In [2]:
import torch
import torch.nn.functional as F
import numpy as np
from functions import *
from models import *

In [3]:
filename = 'input.txt'
#filename = 'wodehouse_right_ho_jeeves.txt'
data_raw = open(filename, 'r').read() # should be simple plain text file
#data_raw = data_raw.lower()
#data_raw = data_raw[0:10009]

table = str.maketrans(dict.fromkeys('ï»¿_\xa0¡§¨©ªÃ´'))
data_raw = data_raw.translate(table)

data, vocab_size, idx_to_char, char_to_idx = data_from_text(data_raw)

#print( 'data_one_hot shape: ', data_one_hot.shape)
#data_raw
#char_to_idx


Data has length 454 and consist of 12 unique characters.


In [4]:
seq_length = 25 # this is how much of the data we sample before updating the params
hidden_size = 100 # size of the hidden state vector
learning_rate = 0.001
num_epochs = 50
 
verbose = True

In [5]:
model = MyLSTM(vocab_size, hidden_size)
model

MyLSTM(
  (lstm): LSTM(12, 100)
  (linear): Linear(in_features=100, out_features=12, bias=True)
)

In [6]:
# initialize parameters

loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train(model, optimizer, loss_fn, num_epochs, data, seq_length)


epoch: 1, loss: 58.174476623535156
epoch: 2, loss: 56.10284423828125
epoch: 3, loss: 54.543941497802734
epoch: 4, loss: 50.54198455810547
epoch: 5, loss: 39.71495819091797
epoch: 6, loss: 32.07414627075195
epoch: 7, loss: 21.78575325012207
epoch: 8, loss: 14.431146621704102
epoch: 9, loss: 8.241966247558594
epoch: 10, loss: 4.934496879577637
epoch: 11, loss: 3.306635856628418
epoch: 12, loss: 2.4206337928771973
epoch: 13, loss: 1.88084077835083
epoch: 14, loss: 1.5358505249023438
epoch: 15, loss: 1.289149284362793
epoch: 16, loss: 1.111598014831543
epoch: 17, loss: 0.9770164489746094
epoch: 18, loss: 0.8730382919311523
epoch: 19, loss: 0.7904462814331055
epoch: 20, loss: 0.7235379219055176
epoch: 21, loss: 0.6684017181396484
epoch: 22, loss: 0.6223416328430176
epoch: 23, loss: 0.5834407806396484
epoch: 24, loss: 0.5502891540527344
epoch: 25, loss: 0.5217976570129395
epoch: 26, loss: 0.49711179733276367
epoch: 27, loss: 0.4755516052246094
epoch: 28, loss: 0.4565868377685547
epoch: 29, l

In [8]:
model.reset_states()

seed_string = 'm'
sample = generate_sample(model, 1000, [char_to_idx[i] for i in seed_string])
''.join([idx_to_char[i] for i in sample])

'y name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is

In [10]:
def print_sample(model, seed_string, length):
    sample = generate_sample(model, length, [char_to_idx[i] for i in seed_string])
    sample = ''.join([idx_to_char[i] for i in sample])
    print(sample)

model.reset_states()
print_sample(model, 'm', 100)

y name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is kemal. my name is 
